In [ ]:
import os 

In [1]:
import sys
print(sys.path)

['c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36\\python36.zip', 'c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36\\DLLs', 'c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36\\lib', 'c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36', '', 'C:\\Users\\Paul PC\\AppData\\Roaming\\Python\\Python36\\site-packages', 'c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36\\lib\\site-packages', 'c:\\users\\paul pc\\appdata\\local\\programs\\python\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Paul PC\\.ipython']


In [4]:
!mkdir data

In [283]:
import os
from collections import defaultdict

import logging
import glob
import re
from PIL import Image
import pandas as pd
import cv2 as cv
import copy as cp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

random_seed = 42
%matplotlib inline

In [2]:
import os

import glob
import re
from PIL import Image
import pandas as pd
import copy as cp
import numpy as np
import matplotlib.pyplot as plt

random_seed = 42
%matplotlib inline

ImportError: Unable to import required dependencies:
numpy: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: DLL load failed: Le module spécifié est introuvable.


In [304]:
pC = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
mC = [33, 34, 35, 36, 37, 38, 39, 40]
dC = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

ob_names="prohibitory\ndanger\nmandatory\nother"

class DataLoader:
    
    def __init__(self,names_path,desc_path,label_path,in_img_path,out_img_path,verbose,list_file_dir,is_4_cat):
        self.verbose = verbose
        self.names_path = names_path
        self.desc_path = desc_path
        self.in_img_path = in_img_path
        self.out_img_path = out_img_path
        self.label_path = label_path
        self.list_file_dir = list_file_dir
        self.gtsdb = pd.read_csv(os.path.abspath(label_path), sep=";",names=
                                 ["img", "x1", "y1", "x2", "y2","id"])
        self.is_4_cat = is_4_cat
        print(self.gtsdb.dtypes)
        self.complete_dataset()
        if(self.is_4_cat):
            self.class2cat()
        print(self.gtsdb.head())
        
    def complete_dataset(self):
        for file_name in glob.glob(os.path.join(self.in_img_path,'*.ppm')):
            name = os.path.split(file_name)[-1]
            if(name not in self.gtsdb.img.tolist()):
                if(self.verbose):
                    print(str.format("adding {0} to dataset",name))
                image = Image.open(file_name)
                self.gtsdb.loc[len(self)] = [name,-1,-1,-1,-1,-1]
            
            
        
    def __getitem__(self,key):
        if(isinstance(key,slice) ):
            indices = key.indices(len(self))
            return [self[ii] for ii in range(*indices)]
        return {'img':self.load_image(key),
                'label':self.gtsdb.iloc[key]['id'],
                'name':self.gtsdb.iloc[key]['img'],
                'x1':self.gtsdb.iloc[key]['x1'],
               'x2':self.gtsdb.iloc[key]['x2'],
               'y1':self.gtsdb.iloc[key]['y1'],
                'y2':self.gtsdb.iloc[key]['y2']}
        
    def __len__(self):
        return len(self.gtsdb)
        
    
    def load_image(self,idx):
        if(self.verbose):
            print("Loading image at {0}".format(self.gtsdb.iloc[idx]['img']))
        return Image.open(os.path.join(self.in_img_path,self.gtsdb.iloc[idx]['img']))
    
    def format_class_name(self,class_name):
        class_name = class_name.lower().replace(' ','-')+'\n'
        if(self.verbose):
            print(class_name)
        return class_name
    
    def dump_class_name(self,path,class_name):
        with open(os.path.abspath(path),'w') as f:
            f.write(class_name+os.linesep)
            
    def dump_classes(self):
        if(self.is_4_cat):
            with open(os.path.abspath(self.names_path), 'w') as names_file:
                names_file.write(ob_names)
            return
        lines = []
        with open(os.path.abspath(self.desc_path),'r') as desc_file:
            lines = desc_file.readlines()
        if(self.verbose):
            print("".join(lines))
        pattern = re.compile('^\d{1,2}\s=\s(.*)')
        search  = lambda l : re.search(pattern,l)
        names =[match.group(1) for match in map(search,lines) if(match is not None) ]
        with open(os.path.abspath(self.names_path), 'w') as names_file:
            for n in names:
                names_file.write(self.format_class_name(n))
   
    def class2cat(self):
#        class2catDict = {}
#        for c,ids in enumerate([pC,mC,dC]):
#            class2catDict.update({idx:c for idx in ids})    
#       print(class2catDict)
#       self.gtsdb.id = self.gtsdb.apply(lambda x:class2catDict[x.id] if x.id is not np.nan else None,axis=1)   
        for index, row in self.gtsdb.iterrows():
            if(row.id in pC):
                self.gtsdb.loc[index,'id']=0
            elif(row.id in mC):
                self.gtsdb.loc[index,'id']=1
            elif(row.id in dC):
                self.gtsdb.loc[index,'id']=2
            elif(row.id is -1 ):
                self.gtsdb.loc[index,'id']=-1
            else:
                self.gtsdb.loc[index,'id']=3
        
    def convert_all(self):
        cache = []
        for im in tqdm(self):
            if(im['name'] not in cache):
                self.ppm2jpg(im['img'],im['name'])
                cache.append(im['name'])
                    
    
    def ppm2jpg(self,img,img_name):
        os.makedirs(self.out_img_path,exist_ok=True)
        img.save(os.path.join(self.out_img_path,os.path.splitext(img_name)[0]+'.jpg'))
    
    
    def get_bb_coordinates(self,example):
        im_height = example['img'].height
        im_width = example['img'].width
        x_center = (example['x2']+example['x1'])/(2.0*im_width)
        y_center = (example['y2']+example['y1'])/(2.0*im_height)
        height = (example['y2']-example['y1'])/im_height
        width = (example['x2']-example['x1'])/im_width
        return{'label':example['label'],
               'x_center':x_center,
               'y_center':y_center,
               'width':width,
               'height':height}
    

    
    def dump_split(self):
        group_by_img = self.gtsdb.groupby('img')
        count = 0
        train_set = []
        test_set = []
        with open(os.path.join(self.list_file_dir,'train.txt'),'w') as train_file, open(os.path.join(self.list_file_dir,'test.txt'),'w') as test_file: 
            for name,group in tqdm(group_by_img):
                base_name=os.path.splitext(name)[0]
                line = os.path.join(self.out_img_path,os.path.splitext(name)[0]+'.jpg')+'\n'
                if(self.verbose):
                    print(str.format("Appending line {0}",line))
                if(int(base_name)<600):
                    train_file.write(line)
                else:
                    test_file.write(line)
                count += 1
        return train_set,test_set
        
    def dump_all(self):
        for img_name in self.gtsdb.img:
            if(self.verbose):
                print(img_name)
            with open(os.path.join(self.out_img_path,os.path.splitext(img_name)[0]+'.txt'),'w') as label_file:
                for index, row in self.gtsdb.loc[self.gtsdb['img'] == img_name].iterrows():
                    example = self[index]
                    if(example['label'] not in ['-1',-1,None,np.nan]):
                        bb_coordinates = self.get_bb_coordinates(example)
                        if(self.verbose):
                            print(example)
                            print(index)
                            print(bb_coordinates)
                        sep = ' '
                        label_file.write(str(bb_coordinates['label'])+sep+str(bb_coordinates['x_center'])+sep+
                                         str(bb_coordinates['y_center'])
                                         +sep+str(bb_coordinates['width'])+sep+str(bb_coordinates['height'])+'\n')
 
    
                

In [240]:

    
def inv_conv_coordinates(x_center,y_center,rel_height,rel_width, im_height,im_width):
    width = rel_width*im_width 
    height = rel_height*im_height
    y1 = y_center*im_height - height//2
    y2 = y_center*im_height + height//2
    x1 = x_center*im_width - width//2
    x2 = x_center*im_width + width//2
    return x1,y1,x2,y2
    
    
def draw_bb(im,relative_center,relative_dimensions,im_dimensions):    
    im = cp.copy(im)
    print(relative_center)
    print(relative_dimensions)
    print(im_dimensions)
    x_center,y_center = relative_center
    height, width = relative_dimensions
    im_height, im_width,_ = im_dimensions
    x1,y1,x2,y2=inv_conv_coordinates(x_center,y_center,height,width, im_height,im_width)
    print(x1,y1,x2,y2)
    return cv.rectangle(im,(int(x1),int(y1)),(int(x2),int(y2)),color=(255,0,0),thickness=2)

def display_image(im):
    plt.figure(figsize=(20,20))
    plt.subplot(1,2,1),plt.imshow(im)
    plt.show()
    
def PIL2Np(im):
    pil_image = im.convert('RGB') 
    open_cv_image = np.array(pil_image)
    return open_cv_image[:, :, ::-1].copy()

def data_split(dataset,validation_split,shuffle_dataset):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_set, val_set = dataset[split:], dataset[:split]
    return train_set,val_set

def IJCNN_data_split(dataset):
    group_by_img = dataset.gtsdb.groupby('img')
    count = 0
    train_set = []
    test_set = []
    for name,group in group_by_img:
        for index,example in group.iterrows():
            print(index)
            if(count<600):
                train_set.append(example)
            else:
                test_set.append(example)
        count += 1
    return train_set,test_set


In [309]:
dataLoader = DataLoader(verbose=False,names_path=os.path.join('data','obj.names'),desc_path=os.path.
           join('FullIJCNN2013','ReadMe.txt'),in_img_path='FullIJCNN2013',out_img_path= os.path.join('data','obj'),
                        label_path=os.path.join('FullIJCNN2013','gt.txt'),list_file_dir='data',is_4_cat=False)
assert len(dataLoader.gtsdb.groupby('img')) == 900

img    object
x1      int64
y1      int64
x2      int64
y2      int64
id      int64
dtype: object
         img   x1   y1    x2   y2  id
0  00000.ppm  774  411   815  446  11
1  00001.ppm  983  388  1024  432  40
2  00001.ppm  386  494   442  552  38
3  00001.ppm  973  335  1031  390  13
4  00002.ppm  892  476  1006  592  39


In [288]:
dataLoader.class2cat()

In [256]:
dataLoader.gtsdb.dtypes


img    object
x1      int64
y1      int64
x2      int64
y2      int64
id      int64
dtype: object

In [298]:
dataLoader.gtsdb.head()

,img,x1,y1,x2,y2,id
0,00000.ppm,774,411,815,446,2
1,00001.ppm,983,388,1024,432,1
2,00001.ppm,386,494,442,552,1
3,00001.ppm,973,335,1031,390,3
4,00002.ppm,892,476,1006,592,1


In [7]:
train_set,test_set = data_split(dataLoader,0.8,True)
IJCNN_train,IJCNN_test = IJCNN_data_split(dataLoader)

In [162]:
img = dataLoader[2]

print(img)
# check that all of the we only have one row per no-object image
assert len(dataLoader.gtsdb.loc[~dataLoader.gtsdb['id'].isin(range(44))].groupby('img')) == len(dataLoader.gtsdb.
                                                                                                loc[~dataLoader.gtsdb['id'].
                                                                                                    isin(range(44))])


{'img': <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1360x800 at 0x1EB1F3975C0>, 'label': 38, 'name': '00001.ppm', 'x1': 386, 'x2': 442, 'y1': 494, 'y2': 552}


In [289]:
dataLoader.gtsdb.tail()


,img,x1,y1,x2,y2,id
1367,00878.ppm,-1,-1,-1,-1,-1
1368,00880.ppm,-1,-1,-1,-1,-1
1369,00883.ppm,-1,-1,-1,-1,-1
1370,00890.ppm,-1,-1,-1,-1,-1
1371,00892.ppm,-1,-1,-1,-1,-1


(800, 1360, 3)
(0.6558823529411765, 0.52875)
(0.055, 0.03529411764705882)
(800, 1360, 3)
868.0 401.00000000000006 916.0 445.00000000000006


In [293]:
# Sanity check
example = dataLoader[896]

im = PIL2Np(example['img'])

print(im.shape)

bb_coordinates = dataLoader.get_bb_coordinates(example)

new_im = draw_bb(im,(bb_coordinates['x_center'],bb_coordinates['y_center']),(bb_coordinates['height'],bb_coordinates['width']),
        im.shape)
display_image(new_im)

In [121]:
dataLoader.convert_all()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1372/1372 [00:44<00:00, 30.97it/s]


In [306]:

dataLoader.dump_all()


In [310]:
dataLoader.dump_classes()

In [308]:
dataLoader.dump_split()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [00:00<00:00, 5246.55it/s]


([], [])

In [290]:
a = dataLoader.gtsdb[['id']].plot.hist(by='id',bins=5)

In [4]:
mapDfxCenter = (dataLoader.gtsdb['x1'] + dataLoader.gtsdb['x2'])/2
mapDfxCenter.head()
mapDfyCenter = (dataLoader.gtsdb['y1'] + dataLoader.gtsdb['y2'])/2


In [5]:
import seaborn as sns

In [18]:
df = pd.DataFrame({'x':list(mapDfxCenter),'y':list(mapDfyCenter)}).dropna()

df.head()

sns.kdeplot(df.x,df.y,shade=True)
